# Homework 1
***

We are going to work with the following dataset: fluid current in a tube.
Some statistics are collected for dataset, incl. mean, skewness, kurtosis, etc. We are predicting flow rate ('tohn/hour'). We need to build confidence and predictive intervals.

In [1]:
%matplotlib inline

import numpy as np
from sklearn import datasets, linear_model, preprocessing, model_selection
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import pandas as pd

In [9]:
df = pd.read_csv('exxsol_data.csv', sep=';', header=(0))
df.head()

,mean,std,skew,kurt,RMS,crest,freq_peak,shan,perm,temp,tohn/hour
0,0.025639,0.946336,-0.034895,-1.491265,0.946674,0.888424,526.01052,9.584955,1.000464,21.14,3.49
1,0.025639,0.946336,-0.034895,-1.491265,0.946674,0.888424,526.01052,9.584955,1.000464,21.14,3.49
2,0.016856,0.952439,-0.037338,-1.504434,0.952579,0.943467,530.01060,9.402718,0.991088,21.14,3.49
3,0.016856,0.952439,-0.037338,-1.504434,0.952579,0.943467,530.01060,9.402718,0.991088,21.14,3.49
4,0.031998,0.972590,-0.056116,-1.546518,0.973106,0.921600,256.00512,9.020444,0.973701,21.14,3.56


There are 10 features and 1 label to predict:

In [3]:
print(df.columns.values)

['mean' 'std' 'skew' 'kurt' 'RMS' 'crest' 'freq_peak' 'shan' 'perm' 'temp'
 'tohn/hour']


In [4]:
y = df['tohn/hour']
freq_temp = df[['freq_peak','temp']]

Physics tells us that flow rate is a function of a frequency peak and temperature.

In [6]:
freq_temp, y = shuffle(freq_temp, y)

# split data into training and testing sets
#from sklearn.model_selection import train_test_split
#train_freq, test_freq, train_y, test_y = train_test_split(freq, y, train_size=0.7, random_state=2)

lr = linear_model.LinearRegression()
predicted = model_selection.cross_val_predict(
    lr, freq_temp, y.to_numpy(), cv=20)
score = model_selection.cross_val_score(lr, freq_temp, y,
                                         scoring='r2',cv=20)

## Q0: Build point estimate for mean r2 score and its deviation

In [11]:
# Your code here
mean_r2 = np.mean(score)
std_r2 = np.std(score)

print(f"Mean R^2 score: {mean_r2:.4f}")
print(f"Standard Deviation of R^2 Scores: {std_r2:.4f}")

Mean R^2 score: 0.8275
Standard Deviation of R^2 Scores: 0.0587


## Q1: Predicted is an array with predictions of the label y. Assuming, that $\sigma = 0.1$, compute 95% confidence and predictive interval for mean squared error. 

In [15]:
# Your code here

mse = np.mean((y - predicted) ** 2)

from scipy.stats import t
n = len(y)
alpha = 0.05
sigma = 0.1

t_critical = t.ppf(1 - alpha / 2, df = n - 1)
margin_of_error_ci = t_critical * (sigma / np.sqrt(n))

ci_lower = mse - margin_of_error_ci
ci_upper = mse + margin_of_error_ci

print(f"Mean Square Error: {mse}")
print(f"95% Confidence Interval: ({ci_lower:.4f}, {ci_upper:.4f})")

Mean Square Error: 1.9352721793461563
95% Confidence Interval: (1.9279, 1.9427)


## Q2:  Compute 95% confidence and predicted intervals for mean squared error, assuming no knowledge about $\sigma$.

In [19]:
# Your code here

residuals = y - predicted

sample_mean_residuals = np.mean(residuals)
sample_std_residuals = np.std(residuals, ddof=1)

mse = np.mean((y - predicted) ** 2)

n = len(y)
alpha = 0.05

t_critical = t.ppf(1 - alpha / 2, df=n - 1)

margin_of_error_ci = t_critical * (sample_std_residuals / np.sqrt(n))

ci_lower = mse - margin_of_error_ci
ci_upper = mse + margin_of_error_ci

print(f"Mean Square Error: {mse}")
print(f"95% Confidence Interval: ({ci_lower:.4f}, {ci_upper:.4f})")

Mean Square Error: 1.9352721793461563
95% Confidence Interval: (1.8320, 2.0386)


We can use additional features and more complex model, e.g. ElasticNet.

In [20]:
y = df['tohn/hour']
X = df.drop(['tohn/hour'],axis=1)
X = preprocessing.scale(X)
X, y = shuffle(X, y)

encv = linear_model.ElasticNetCV(cv=10,max_iter=3000, n_alphas=10)
predicted_encv = model_selection.cross_val_predict(
    encv, X, y.to_numpy(), cv=20)
score_encv = model_selection.cross_val_score(encv,X, y.to_numpy(),
                                         scoring='r2',cv=20)

## Q3:  Compute 95% confidence interval for difference in means of mean squared error between 2 models, assuming no knowledge about $\sigma$.

In [24]:
# Your code here
mse_lr = mse
mse_encv = np.mean((y - predicted_encv) ** 2)

delta = mse_lr - mse_encv

residuals_lr = residuals
residuals_encv = y - predicted_encv

var_lr = np.var(residuals_lr, ddof=1)
var_encv = np.var(residuals_encv, ddof=1)

n = len(y)

se_delta = np.sqrt(var_lr / n + var_encv / n)

alpha = 0.05

t_critical = t.ppf(1 - alpha / 2, df=n - 1)
margin_of_errors = t_critical * se_delta

ci_lower = delta - margin_of_errors
ci_upper = delta + margin_of_errors

print(f"difference: {delta:.4f}")
print(f"95% Confidence Interval for Difference in MSEs: ({ci_lower:.4f}, {ci_upper:.4f})")

difference: 1.8758
95% Confidence Interval for Difference in MSEs: (1.7709, 1.9807)


## Q4: Implement UCB1

In [27]:
import numpy as np

class UCB1:
    def __init__(self, num_arms: int):
        self.num_arms = num_arms
        self.total_pulls = 0
        self.arm_pulls = np.zeros(num_arms)
        self.rewards = np.zeros(num_arms)

    def select_arm(self):
        if self.total_pulls < self.num_arms:
            return self.total_pulls

        # self.rewards / (self.arm_pulls + 1e-6) -- cредний доход со всех ручек
        # np.sqrt(2 * np.log(self.total_pulls) / (self.arm_pulls + 1e-6)) -- сколько получим
        ucb_values = (
            self.rewards / (self.arm_pulls + 1e-6) + 
            np.sqrt(2 * np.log(self.total_pulls) / (self.arm_pulls + 1e-6))
        )

        return np.argmax(ucb_values)

    def update(self, arm, reward):
        self.total_pulls += 1
        self.arm_pulls[arm] += 1
        self.rewards[arm] += reward